# Install and Load Libraries

In [1]:
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/pip-install.py

Cloning into 'rapidsai-csp-utils'...
remote: Enumerating objects: 596, done.
remote: Counting objects: 100% (162/162), done.
remote: Compressing objects: 100% (80/80), done.
remote: Total 596 (delta 128), reused 82 (delta 82), pack-reused 434 (from 3)
Receiving objects: 100% (596/596), 195.77 KiB | 16.31 MiB/s, done.
Resolving deltas: 100% (302/302), done.
Installing RAPIDS remaining 25.04 libraries
Using Python 3.11.13 environment at: /usr
Resolved 173 packages in 11.32s
 Downloaded cuproj-cu12
 Downloaded ucx-py-cu12
 Downloaded shapely
 Downloaded datashader
 Downloaded cuspatial-cu12
 Downloaded libkvikio-cu12
 Downloaded dask
 Downloaded cudf-cu12
 Downloaded pylibcugraph-cu12
 Downloaded rmm-cu12
 Downloaded cucim-cu12
 Downloaded bokeh
 Downloaded cugraph-cu12
 Downloaded librmm-cu12
 Downloaded pylibcudf-cu12
 Downloaded cuml-cu12
 Downloaded libraft-cu12
 Downloaded libcuspatial-cu12
 Downloaded libcuml-cu12
 Downloaded raft-dask-cu12
 Downloaded libcudf-cu12
 Downloaded libcu

In [2]:
!pip install kaggle --quiet

In [5]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [6]:
!kaggle datasets download -d sobhanmoosavi/us-accidents

Dataset URL: https://www.kaggle.com/datasets/sobhanmoosavi/us-accidents
License(s): CC-BY-NC-SA-4.0
 96% 629M/653M [00:12<00:01, 12.9MB/s]
100% 653M/653M [00:12<00:00, 55.4MB/s]


In [7]:
!unzip us-accidents.zip

Archive:  us-accidents.zip
  inflating: US_Accidents_March23.csv  


In [8]:
import cudf
cudf.__version__

'25.04.00'

# Data Extraction

In [9]:
import cudf

df = cudf.read_csv("/content/US_Accidents_March23.csv")

In [ ]:
df.info()

<class 'cudf.core.dataframe.DataFrame'>
RangeIndex: 7728394 entries, 0 to 7728393
Data columns (total 46 columns):
 #   Column                 Dtype
---  ------                 -----
 0   ID                     object
 1   Source                 object
 2   Severity               int64
 3   Start_Time             object
 4   End_Time               object
 5   Start_Lat              float64
 6   Start_Lng              float64
 7   End_Lat                float64
 8   End_Lng                float64
 9   Distance(mi)           float64
 10  Description            object
 11  Street                 object
 12  City                   object
 13  County                 object
 14  State                  object
 15  Zipcode                object
 16  Country                object
 17  Timezone               object
 18  Airport_Code           object
 19  Weather_Timestamp      object
 20  Temperature(F)         float64
 21  Wind_Chill(F)          float64
 22  Humidity(%)            float64
 23  

In [10]:
df.describe()

,Severity,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Speed(mph),Precipitation(in)
count,7.728394e+06,7.728394e+06,7.728394e+06,4.325632e+06,4.325632e+06,7.728394e+06,7.564541e+06,5.729375e+06,7.554250e+06,7.587715e+06,7.551296e+06,7.157161e+06,5.524808e+06
mean,2.212384e+00,3.620119e+01,-9.470255e+01,3.626183e+01,-9.572557e+01,5.618420e-01,6.166329e+01,5.825105e+01,6.483104e+01,2.953899e+01,9.090376e+00,7.685490e+00,8.407000e-03
std,4.875310e-01,5.076079e+00,1.739176e+01,5.272905e+00,1.810793e+01,1.776811e+00,1.901365e+01,2.238983e+01,2.282097e+01,1.006190e+00,2.688316e+00,5.424983e+00,1.102250e-01
min,1.000000e+00,2.455480e+01,-1.246238e+02,2.456601e+01,-1.245457e+02,0.000000e+00,-8.900000e+01,-8.900000e+01,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,2.000000e+00,3.339963e+01,-1.172194e+02,3.346207e+01,-1.177543e+02,0.000000e+00,4.900000e+01,4.300000e+01,4.800000e+01,2.937000e+01,1.000000e+01,4.600000e+00,0.000000e+00
50%,2.000000e+00,3.582397e+01,-8.776662e+01,3.618350e+01,-8.802789e+01,3.000000e-02,6.400000e+01,6.200000e+01,6.700000e+01,2.986000e+01,1.000000e+01,7.000000e+00,0.000000e+00
75%,2.000000e+00,4.008496e+01,-8.035368e+01,4.017892e+01,-8.024709e+01,4.640000e-01,7.600000e+01,7.500000e+01,8.400000e+01,3.003000e+01,1.000000e+01,1.040000e+01,0.000000e+00
max,4.000000e+00,4.900220e+01,-6.711317e+01,4.907500e+01,-6.710924e+01,4.417500e+02,2.070000e+02,2.070000e+02,1.000000e+02,5.863000e+01,1.400000e+02,1.087000e+03,3.647000e+01


### drop columns

In [11]:
# df["Country"].unique()

df.drop(columns=["Country","ID","Description"], inplace=True)



# bool to num
# cheang astype to uint8

In [12]:
import numpy as np

In [38]:
bull_cilumes = ["Severity","Humidity(%)","Amenity","Turning_Loop","Bump","Crossing","Give_Way","Junction","No_Exit","Railway","Roundabout","Station","Stop","Traffic_Calming","Traffic_Signal"]
for cul in bull_cilumes:
   df[cul] = df[cul].astype(np.uint8)
df.info()

<class 'cudf.core.dataframe.DataFrame'>
RangeIndex: 7728394 entries, 0 to 7728393
Data columns (total 43 columns):
 #   Column                 Dtype
---  ------                 -----
 0   Source                 object
 1   Severity               uint8
 2   Start_Time             object
 3   End_Time               object
 4   Start_Lat              float32
 5   Start_Lng              float32
 6   End_Lat                float32
 7   End_Lng                float32
 8   Distance(mi)           float32
 9   Street                 uint32
 10  City                   uint16
 11  County                 uint16
 12  State                  uint8
 13  Zipcode                object
 14  Timezone               object
 15  Airport_Code           object
 16  Weather_Timestamp      object
 17  Temperature(F)         float32
 18  Wind_Chill(F)          float32
 19  Humidity(%)            uint8
 20  Pressure(in)           float32
 21  Visibility(mi)         float32
 22  Wind_Direction         object
 23  W

#citi change

In [14]:
import sklearn

In [15]:
df["County"].unique()

0       Montgomery
1         Franklin
2         Clermont
3          Licking
4        Fairfield
           ...    
1866        Mcminn
1867      Glascock
1868         Woods
1869      Mellette
1870          Ness
Name: County, Length: 1871, dtype: object

In [16]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
item = ["Street","City","County","State"]
for i in item:
  df[i] = encoder.fit_transform(df[i].to_numpy())



In [17]:
df["State"] = df["State"].astype(np.uint8)
df["County"] = df["County"].astype(np.uint16)
df["City"] = df["City"].astype(np.uint16)
df["Street"] = df["Street"].astype(np.uint32)

# Twilight

In [18]:
l = ["Sunrise_Sunset","Civil_Twilight","Nautical_Twilight","Astronomical_Twilight"]
for i in l:
  df[i] = encoder.fit_transform(df[i].to_numpy()).astype(np.uint8)

In [68]:
#df["Weather_Condition"].value_counts()
df["Weather_Timestamp"].head(20)

0     2016-02-08 05:58:00
1     2016-02-08 05:51:00
2     2016-02-08 06:56:00
3     2016-02-08 07:38:00
4     2016-02-08 07:53:00
5     2016-02-08 07:51:00
6     2016-02-08 07:56:00
7     2016-02-08 07:56:00
8     2016-02-08 07:58:00
9     2016-02-08 08:28:00
10    2016-02-08 07:50:00
11    2016-02-08 08:28:00
12    2016-02-08 08:28:00
13    2016-02-08 08:56:00
14    2016-02-08 08:28:00
15    2016-02-08 08:28:00
16    2016-02-08 08:58:00
17    2016-02-08 08:56:00
18    2016-02-08 09:38:00
19    2016-02-08 09:56:00
Name: Weather_Timestamp, dtype: object

# some maping

In [62]:
df["Source"] = df["Source"].map({"Source1":1, "Source2":2, "Source3":3})

df["Timezone"] = df["Timezone"].map({"US/Eastern":1, "US/Pacific":2, "US/Central":3, "US/Mountain":4})


for cul in ["Source","Timezone"]:
  df[cul] = df[cul].astype(np.uint8)




# weder
#cheange astype to float32

In [39]:
Gi = ["Wind_Speed(mph)","Precipitation(in)","Start_Lat","Start_Lng","End_Lat","End_Lng","Distance(mi)","Temperature(F)","Wind_Chill(F)","Pressure(in)","Visibility(mi)"]
for cul in Gi:
  df[cul] = df[cul].astype(np.float32)

In [80]:
df.info()

<class 'cudf.core.dataframe.DataFrame'>
RangeIndex: 7728394 entries, 0 to 7728393
Data columns (total 43 columns):
 #   Column                 Dtype
---  ------                 -----
 0   Source                 uint8
 1   Severity               uint8
 2   Start_Time             object
 3   End_Time               object
 4   Start_Lat              float32
 5   Start_Lng              float32
 6   End_Lat                float32
 7   End_Lng                float32
 8   Distance(mi)           float32
 9   Street                 uint32
 10  City                   uint16
 11  County                 uint16
 12  State                  uint8
 13  Zipcode                object
 14  Timezone               uint8
 15  Airport_Code           object
 16  Weather_Timestamp      object
 17  Temperature(F)         float32
 18  Wind_Chill(F)          float32
 19  Humidity(%)            uint8
 20  Pressure(in)           float32
 21  Visibility(mi)         float32
 22  Wind_Direction         object
 23  Win

# To Date time

In [76]:
import pandas as pd

In [79]:
cols = ["Start_Time","End_Time","Weather_Timestamp"]

for col in cols:
  df[cul] = df[cul].astype(np.datetime64)


TypeError: Unsupported type datetime64